<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/NLP/Langchain/langchain_IO_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain CrashCourse

## Set up

In [2]:
%%capture
!pip install peft==0.4.0 trl==0.4.7 !pip install Pillow>=10.1.0 torch>=2.1.2 torchvision>=0.16.2 transformers>=4.40.0 sentencepiece>=0.1.99 accelerate>=0.30.1 bitsandbytes>=0.43.1

In [3]:
%%capture
!pip install langchain langchain-community langchain-core

## Data downloads

In [4]:
!kaggle datasets download -d PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
!unzip amazon-reviews-unlocked-mobile-phones.zip

Dataset URL: https://www.kaggle.com/datasets/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
License(s): CC0-1.0
 92% 30.0M/32.6M [00:01<00:00, 25.4MB/s]
100% 32.6M/32.6M [00:01<00:00, 20.8MB/s]
Archive:  amazon-reviews-unlocked-mobile-phones.zip
  inflating: Amazon_Unlocked_Mobile.csv  


## Imports

In [5]:
import numpy as np
import pandas as pd
import torch
import transformers
import warnings

from google.colab import userdata
from huggingface_hub import login
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from pprint import pprint
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, pipeline


warnings.filterwarnings("ignore")
hf = userdata.get('hugging_face')
login(hf)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load data

In [6]:
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
df.dropna(inplace=True)

In [5]:
df.sample(5)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
239601,Life Pure Mini Quad Band Unlocked (White),BLU,120.00,5,Everything was perfect,0.0
345987,Samsung Galaxy S4 SGH-I337 Unlocked GSM Smartp...,Samsung,288.89,5,"Okay so I was not a fan of samsung, two many f...",0.0
368677,Samsung Galaxy S6 SM-G920i Factory Unlocked Ce...,samsung,429.99,1,I don't know if I just so happened to get a ba...,4.0
318235,Samsung Galaxy J5 SSJ500MGD Factory Unlocked D...,Samsung,197.95,4,UPDATE 5.2.16I had my phone for almost a month...,6.0
176925,CNPGD MultiFunction All-in-1 Watch Cell Phone ...,CNPGD,79.99,4,Very good,0.0


## Load model

In [12]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [13]:
model_name = 'meta-llama/Llama-2-7b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, load_in_4bit=True)

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id)
print(output[0]['generated_text'])

Outputs something like this:
pretty unstructured

'I need to analyze amazon product reviews from Amazon_Unlocked_Mobile '
 'dataset.could you suggest how to do it best?\n'
 '\n'
 'Answer: Yes, I can help you with that! Analyzing Amazon product reviews from '
 'the Amazon_Unlocked_Mobile dataset can be a challenging task, but here are '
 'some steps you can follow to do it effectively:\n'
 '\n'
 '1. Preprocessing:\n'
 '\t* Remove stop words: Remove common words like "the", "a", "and", etc. that '
 "don't add much value to the review.\n"
 '\t* Lemmatize words: Convert words to their base form (e.g., "running" '
 'becomes "run") to reduce the dimensionality of the data.\n'
 '\t* Remove special characters and punctuation: Remove any special characters '
 'or punctuation marks from the reviews, as they may not be relevant to the '
 'analysis.\n'
 '2. Feature extraction:\n'
 '\t* Sentiment analysis: Use a sentiment analysis tool or a machine learning '
 'model to extract the sentiment of the reviews. This can help you identify '
 'whether the reviews are generally positive, negative, or neutral.\n'
 '\t* Aspect-based sentiment analysis: Identify the different aspects of the '
 'product that are mentioned in the reviews, such as the quality of the '
 'materials, the design, the performance, etc. Analyze the sentiment of each '
 'aspect to understand how customers perceive it.\n'
 '\t* Named entity recognition: Identify specific entities mentioned in the '
 "reviews, such as the product name, the company name, or the customer's name. "
 'This can help you understand the context of the reviews.\n'
 '3. Clustering:\n'
 '\t* K-means clustering: Use k-means clustering to group similar reviews '
 'together based on their sentiment and aspect-based sentiment analysis. This '
 'can help you identify common themes or issues that customers are discussing '
 'in their reviews.\n'
 '\t* Hierarchical clustering: Use hierarchical clustering to create a '
 'dendrogram that shows the similarity between the reviews. This can help you '
 'identify clusters of reviews that are similar in sentiment or aspect-based '
 'sentiment analysis.\n'
 '4. Visualization:\n'
 '\t* Bar chart or histogram: Use a bar chart or histogram to visualize the '
 'distribution of sentiment in the reviews. This can help you identify whether '
 'the reviews are generally positive, negative, or neutral.\n'
 '\t* Scatter plot: Use a scatter plot to visualize the relationship between '
 'different aspects of the product and the sentiment of the reviews. This can '
 'help you identify which aspects of the product are most important for '
 'customer'